In [25]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
import os
os.chdir("/home/furfa/work/ai-academy2019/notebooks")
pd.set_option("max_columns", 10000)

%pylab inline
import seaborn as sns

import warnings
warnings.simplefilter("ignore")

import json
from IPython import display

from tqdm import tqdm
from Scripts.saving import submit

Populating the interactive namespace from numpy and matplotlib


In [26]:
base_dir = ".."
input_dir = os.path.join(base_dir, "input")
pred_dir = os.path.join(base_dir, "predictions")
processed_dir = os.path.join(input_dir, "processed")
!tree {input_dir}

../input
|-- ai-academy-2019-skill-prediction-data-csv-v1.zip
|-- ai-academy-2019-skill-prediction-data-jsonlines.zip
|-- dota2_abilities.csv
|-- dota2_heroes.csv
|-- dota2_items.csv
|-- dota2_skill_test.csv
|-- dota2_skill_test.jsonlines
|-- dota2_skill_train.csv
|-- dota2_skill_train.jsonlines
`-- processed
    |-- test_proc.csv
    |-- test_team.csv
    |-- train_proc.csv
    `-- train_team.csv

1 directory, 13 files


In [27]:
# aditional
abilities = pd.read_csv( os.path.join(input_dir, 'dota2_abilities.csv') )
items = pd.read_csv( os.path.join(input_dir, 'dota2_items.csv') )
heroes = pd.read_csv( os.path.join(input_dir, 'dota2_heroes.csv') )
# Main dataset
train = pd.read_csv( os.path.join(input_dir, 'dota2_skill_train.csv'), index_col=0 )
test = pd.read_csv( os.path.join(input_dir, 'dota2_skill_test.csv'), index_col=0 )

new_features_train = pd.read_csv( os.path.join(processed_dir, 'train_proc.csv'), index_col=0 )
new_features_test = pd.read_csv( os.path.join(processed_dir, 'test_proc.csv'), index_col=0 )

team_train = pd.read_csv( os.path.join(processed_dir, 'train_team.csv'), index_col=0 )
team_test = pd.read_csv( os.path.join(processed_dir, 'test_team.csv'), index_col=0 )

In [28]:
print("Deleted", set(train.columns).difference( set(test.columns) ) )

y_train = train.skilled
train.drop(["skilled"], 1, inplace = True)

Deleted {'skilled'}


In [29]:
# Transformation
def apply(func, columns):
    global train, test
    for c in tqdm(columns):
        train[c] = train[c].apply(func)
        test[c] = test[c].apply(func)


def tr(x):
    if x=='dire':
        return 0.0
    elif x=='radiant':
        return 1.0
apply(tr, ["player_team", "winner_team"])

100%|██████████| 2/2 [00:00<00:00, 15.04it/s]


In [30]:
# New features
train = pd.concat((train, new_features_train, team_train), axis=1)
test = pd.concat((test, new_features_test, team_test), axis=1)

ntrain = train.shape[0]
ntest = test.shape[0]
all_data = pd.concat((train, test), axis=0)

In [34]:
all_data['_win'] = all_data['player_team'] == all_data['winner_team']
all_data['_kda'] = (all_data.kills + all_data.assists)/ all_data.deaths
all_data['_duration_min'] = all_data.duration / 60
all_data['__gold_dame'] = all_data['_duration_min'] * all_data.gold_per_min
all_data['__xp_dame'] = all_data['_duration_min'] * all_data.xp_per_min
all_data['__free_gold'] = all_data['__gold_dame'] - all_data.gold_spent

In [35]:
all_data["radiant_win_%"] = abs( all_data["radiant_win_%"] - (1.0 * all_data.player_team) )

In [36]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [40]:
from sklearn.model_selection import cross_val_score
import catboost as cb
import lightgbm as lgb

In [41]:
cross_val_score(
    lgb.LGBMClassifier(),
    train,
    y_train,
    scoring="accuracy",
    cv = 4,
    verbose=1
)

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   13.1s finished


array([0.76158436, 0.75893143, 0.76140505, 0.76084431])

In [ ]:
model = lgb.LGBMClassifier().fit(train, y_train)